In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!git clone https://github.com/leftthomas/SRGAN.git

fatal: destination path 'SRGAN' already exists and is not an empty directory.


In [2]:
%cd "/content/SRGAN"

/content/SRGAN


In [ ]:
!python train.py --crop_size 32 --num_epochs 20

# generator parameters: 734219
# discriminator parameters: 5215425
[1/20] Loss_D: 0.8138 Loss_G: 0.0260 D(x): 0.8424 D(G(z)): 0.6579: 100% 64/64 [00:54<00:00,  1.18it/s]
[converting LR images to SR images] PSNR: 17.9301 dB SSIM: 0.5507: 100% 1026/1026 [02:46<00:00,  6.17it/s]
tcmalloc: large alloc 5909766144 bytes == 0x7fad5ce14000 @  0x7fb14a2a0b6b 0x7fb14a2c0379 0x7fb0eed5192e 0x7fb0eed53946 0x7fb12712fbb3 0x7fb126be417a 0x7fb126be5293 0x7fb1270e5edf 0x7fb126c736a6 0x7fb126c747b0 0x7fb126ecb1c9 0x7fb12686c689 0x7fb126faa919 0x7fb126c7609b 0x7fb127060a08 0x7fb12686c689 0x7fb126faa709 0x7fb126c6bd1d 0x7fb12706adc8 0x7fb12686c689 0x7fb126faab29 0x7fb128ce4347 0x7fb12686c689 0x7fb126faab29 0x7fb1366445b6 0x50a7f5 0x50c1f4 0x507f24 0x50b053 0x634dd2 0x634e87
^C


In [4]:
import PIL
import numpy as np

In [9]:
import cv2,os
path="/content/gdrive/My Drive/SR/finished/valid/dataraw/hires"
for file in os.listdir(path):
    img=PIL.Image.open(os.path.join(path,file))
    shp=np.array(img).shape
    if len(shp) is not 3 or shp[2]>3 or shp[2]==1:
        print("deleting ", file)
        os.remove(os.path.join(path,file))

In [3]:
import argparse
import os
from math import log10
import torch
import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

import pytorch_ssim
from data_utils import TrainDatasetFromFolder, ValDatasetFromFolder, display_transform
from loss import GeneratorLoss
from model import Generator, Discriminator

if 1 == 1:
    #opt = parser.parse_args()
    
    CROP_SIZE = 32
    UPSCALE_FACTOR = 4
    NUM_EPOCHS = 5
    
    train_set = TrainDatasetFromFolder('/content/gdrive/My Drive/SR/finished/train/dataraw/hires', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
    val_set = ValDatasetFromFolder('/content/gdrive/My Drive/SR/finished/test', upscale_factor=UPSCALE_FACTOR)
    train_loader = DataLoader(dataset=train_set, num_workers=0, batch_size=32, shuffle=True)
    val_loader = DataLoader(dataset=val_set, num_workers=0, batch_size=1, shuffle=False)
    
    netG = Generator(UPSCALE_FACTOR)
    print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
    netD = Discriminator()
    print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))
    
    generator_criterion = GeneratorLoss()
    
    if torch.cuda.is_available():
        netG.cuda()
        netD.cuda()
        generator_criterion.cuda()
    
    optimizerG = optim.Adam(netG.parameters())
    optimizerD = optim.Adam(netD.parameters())
    
    results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}
    
    for epoch in range(1, NUM_EPOCHS + 1):
        train_bar = tqdm(train_loader)
        running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}
        torch.autograd.set_detect_anomaly(True)
        netG.train()
        netD.train()
        for data, target in train_bar:
            g_update_first = True
            batch_size = data.size(0)
            running_results['batch_sizes'] += batch_size
    
            ############################
            # (1) Update D network: maximize D(x)-1-D(G(z))
            ###########################
            real_img = Variable(target)
            if torch.cuda.is_available():
                real_img = real_img.cuda()
            z = Variable(data)
            if torch.cuda.is_available():
                z = z.cuda()
            fake_img = netG(z)
    
            netD.zero_grad()
            real_out = netD(real_img).mean()
            fake_out = netD(fake_img).mean()
            d_loss = 1 - real_out + fake_out
            d_loss.backward(retain_graph=True)
            optimizerD.step()
    
            ############################
            # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
            ###########################
            netG.zero_grad()
            fake_img = netG(z)
            fake_out = netD(fake_img).mean()
            g_loss = generator_criterion(fake_out, fake_img, real_img)
            g_loss.backward()
            
            fake_img = netG(z)
            fake_out = netD(fake_img).mean()
            
            
            optimizerG.step()

            # loss for current batch before optimization 
            running_results['g_loss'] += g_loss.item() * batch_size
            running_results['d_loss'] += d_loss.item() * batch_size
            running_results['d_score'] += real_out.item() * batch_size
            running_results['g_score'] += fake_out.item() * batch_size
    
            train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f' % (
                epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
                running_results['g_loss'] / running_results['batch_sizes'],
                running_results['d_score'] / running_results['batch_sizes'],
                running_results['g_score'] / running_results['batch_sizes']))
    
        netG.eval()
        out_path = 'training_results/SRF_' + str(UPSCALE_FACTOR) + '/'
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        
        with torch.no_grad():
            val_bar = tqdm(val_loader)
            valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
            val_images = []
            count=0
            for val_lr, val_hr_restore, val_hr in val_bar:
                #if count>400:
                  #print("breaking out as counter reached ",count)
                  #break
                batch_size = val_lr.size(0)
                #count=count+1
                valing_results['batch_sizes'] += batch_size
                lr = val_lr
                hr = val_hr
                if torch.cuda.is_available():
                    lr = lr.cuda()
                    hr = hr.cuda()
                sr = netG(lr)
        
                batch_mse = ((sr - hr) ** 2).data.mean()
                valing_results['mse'] += batch_mse * batch_size
                batch_ssim = pytorch_ssim.ssim(sr, hr).item()
                valing_results['ssims'] += batch_ssim * batch_size
                valing_results['psnr'] = 10 * log10((hr.max()**2) / (valing_results['mse'] / valing_results['batch_sizes']))
                valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
                val_bar.set_description(
                    desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                        valing_results['psnr'], valing_results['ssim']))
        
                val_images.extend(
                    [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                     display_transform()(sr.data.cpu().squeeze(0))])
            val_images = torch.stack(val_images)
            val_images = torch.chunk(val_images, val_images.size(0) // 15)
            val_save_bar = tqdm(val_images, desc='[saving training results]')
            index = 1
            for image in val_save_bar:
                image = utils.make_grid(image, nrow=3, padding=5)
                utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
                index += 1
    
        # save model parameters
        torch.save(netG.state_dict(), 'epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
        torch.save(netD.state_dict(), 'epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
        # save loss\scores\psnr\ssim
        results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
        results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
        results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
        results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
        results['psnr'].append(valing_results['psnr'])
        results['ssim'].append(valing_results['ssim'])
    
        if epoch % 10 == 0 and epoch != 0:
            out_path = 'statistics/'
            data_frame = pd.DataFrame(
                data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                      'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
                index=range(1, epoch + 1))
            data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')


# generator parameters: 734219
# discriminator parameters: 5215425


[1/5] Loss_D: 0.9832 Loss_G: 0.0217 D(x): 0.6136 D(G(z)): 0.5883: 100%|██████████| 128/128 [02:17<00:00,  1.08s/it]
[converting LR images to SR images] PSNR: 17.6274 dB SSIM: 0.5802: 100%|██████████| 14/14 [00:11<00:00,  1.20it/s]
[saving training results]: 100%|██████████| 2/2 [00:03<00:00,  1.67s/it]
[2/5] Loss_D: 1.0019 Loss_G: 0.0125 D(x): 0.7246 D(G(z)): 0.7251: 100%|██████████| 128/128 [02:14<00:00,  1.05s/it]
[converting LR images to SR images] PSNR: 19.6375 dB SSIM: 0.6234: 100%|██████████| 14/14 [00:04<00:00,  2.93it/s]
[saving training results]: 100%|██████████| 2/2 [00:03<00:00,  1.64s/it]
[3/5] Loss_D: 1.0001 Loss_G: 0.0093 D(x): 0.9736 D(G(z)): 0.9751: 100%|██████████| 128/128 [02:15<00:00,  1.06s/it]
[converting LR images to SR images] PSNR: 17.9070 dB SSIM: 0.6625: 100%|██████████| 14/14 [00:04<00:00,  2.95it/s]
[saving training results]: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]
[4/5] Loss_D: 1.0000 Loss_G: 0.0080 D(x): 0.9999 D(G(z)): 0.9999: 100%|██████████| 128/1